In [10]:
#%matplotlib widget
import numpy as np
import astropy.units as u
from astropy import constants as const
from astropy.io import fits
import matplotlib.pyplot as plt
from specmatchemp import spectrum
import specmatchemp.library
from specmatchemp.specmatch import SpecMatch

## Compute AMD using SpecMatch-Emp for GALAH spectra

In [11]:
# Load spectral library (e.g. Mg b region)
lib = specmatchemp.library.read_hdf(wavlim=[5140,5200])

# Load your observed GALAH spectrum, then crop to library wavelength range
galah_spec = spectrum.read_hires_fits('/datax/scratch/emmay/galah_spectra/161116001701343_R.fits').cut(5130, 5210)
galah_spec.name = 'MyGALAHstar'

# Initialize SpecMatch with your spectrum and the library
sm = SpecMatch(galah_spec, lib)

# Shift your spectrum to library rest wavelengths
sm.shift()

# Perform the spectral match
sm.match(wavlim=(5140,5200))

# Compute a linear combination of best matches for best fit
sm.lincomb()

# Access derived stellar parameters
print("Teff:", sm.results['Teff'])
print("Radius:", sm.results['radius'])
print("[Fe/H]:", sm.results['feh'])

# Calculate residuals between observed and best-fit model
residuals = galah_spec.flux - sm.lincomb_spectrum.flux


Reading library from /home/emmay/.specmatchemp/library.h5
Shifting spectrum
Shifting spectrum MyGALAHstar


IndexError: index 0 is out of bounds for axis 1 with size 0

In [3]:
#Zuckerman (9)
def calc_laser_l(alpha, AMD, lambda_l, d_E, epsilon, del_t, N_obs, A_APF, d_T):
    numerator = 1.22**2*alpha*np.pi*const.h*const.c*AMD*lambda_l*d_E**2
    denominator = 4*epsilon*del_t*N_obs*A_APF*d_T**2
    return numerator/denominator

### APF Laser Detection Threshold

In [4]:
# N_obs=5?
calc_laser_l(alpha=6, AMD=414.78, lambda_l=(5000*u.AA).to(u.m), d_E=(78.47*u.lyr).to(u.m), epsilon=0.05, del_t=730*u.s, N_obs=3, A_APF=4.53*u.m**2, d_T=2.4*u.m).to('kW')

<Quantity 55.73746051 kW>

### GALAH Laser Detection Threshold

In [ ]:
# N_obs=5?
calc_laser_l(alpha=6, AMD=414.78, lambda_l=(5000*u.AA).to(u.m), d_E=(100*u.pc).to(u.m), epsilon=0.05, del_t=730*u.s, N_obs=1, A_APF=4.53*u.m**2, d_T=2.4*u.m).to('kW')